# ReadMe
The objective of this project is to read the National Vulnerability Database's (NVD) of Common exposures and vulnerabilites (CVES) and create a tool 
that is used as a part of the data pipeline to determine how similar CVE's are to one another. 

The first goal to such a pipeline is to clean the description data, tokenize the data. The second goal is tokenize and read the data that will be used as a training label. 

In [18]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: ntp 1.2.0.2021-06-17T05-15-04Z has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of ntp or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [74]:
import os 
import pyspark
import urllib.request
import zipfile
from pyspark.sql import SparkSession
from pyspark.sql.functions import  col, concat_ws, explode, countDistinct, udf, split
from pyspark.sql.types import StringType, ArrayType


In [75]:
# Load the dataset if it doesn't already exist. 
data_dir = "data"
os.makedirs(data_dir, exist_ok=True) 
fileUrls = [
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2002.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2003.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2004.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2005.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2006.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2007.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2008.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2009.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2010.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2011.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2012.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2013.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2014.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2015.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2016.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2017.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2018.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2019.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2020.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2021.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2022.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2023.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2024.json.zip',
        'https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-recent.json.zip'
    ]

# Iterate through each URL
for url in fileUrls:
    filename = url.split("/")[-1]
    outputfile = os.path.join(data_dir, filename)
    checkfile = os.path.join(data_dir, os.path.splitext(filename)[0])

    # Check if the file already exists
    if not os.path.exists(checkfile):
        # Download the file
        urllib.request.urlretrieve(url, outputfile)
        print(f"Downloaded: {filename}")

        # Extract the file
        with zipfile.ZipFile(outputfile, "r") as zip_ref:
            zip_ref.extractall(data_dir)

        # Delete the original zip file
        os.remove(outputfile)



In [85]:
spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("voltcve")
        .config("spark.driver.host", "127.0.0.1")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .config("spark.default.parallelism", 8)
        .getOrCreate()
)
# read the data
cves = spark.read.option("multiline", "true").json("data/nvdcve-1.1-2020.json")

# Manipulate the data to be more usable 
exploded = cves.select(explode(col("CVE_Items")).alias("cves"))

# Now 'exploded' contains individual rows for each 'CVE_Item'
exploded.show()

descr_df = exploded.select(col("cves.cve.CVE_data_meta.ID").alias("id"),
          col("cves.cve.description.description_data.value").alias("description"));

descr_df = descr_df.withColumn("description_single", concat_ws(" ", descr_df["description"]))


doc_count = descr_df.selectExpr("count(distinct id)").first()[0]
print("Number of docs: {}".format(doc_count))

+--------------------+
|                cves|
+--------------------+
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
|{{4.0, [{[], [{cp...|
+--------------------+
only showing top 20 rows



Number of docs: 20453


In [93]:
@udf
def string_cleaner(input_str):
    # 1. Replace all "." or ':' followed by whitespace with an empty string.
    # a. Remove ending periods.
    # 2. Remove trademark, rights.
    # 3. Grab cotent in parentheses only.
    # 4. Remove some punctuation.
    return input_str.replace("[.:\\,]+\\s+|\\.$|\\'|\\(TM\\)|\\(R\\)|\\(|\\)|\\\"", " ").strip().lower()

test = "Beautiful day."
print(string_cleaner(test))

clean_tokens = descr_df.withColumn("tokens", explode(split(string_cleaner(col("description_single")), " " )))
clean_tokens = clean_tokens.select("id", "tokens")
clean_tokens = clean_tokens.filter(col("tokens").isNotNull())

# Show the resulting DataFrame
descr_df.show()

clean_tokens.show()


AnalysisException: Syntax error in attribute name: Beautiful day..

In [ ]:
spark.stop()